In [43]:
import pandas as pd
import openai
import weaviate
import os
import streamlit as st
from dotenv import load_dotenv


df_base = pd.read_excel('Base.xlsx', skiprows = 2, header = 1)
df_base = df_base.iloc[:, :9]
# df_base = df_base.dropna().reset_index(drop=True)
df_base = df_base.fillna("Não informado")


renomear = {"EMPRESA": "EMPRESA",
            "SEGMENTO DE MERCADO": "SEGMENTO_MERCADO",
            "PROCESSO": "PROCESSO",
            "ATIVIDADE RELACIONADA": "ATIVIDADE_RELACIONADA",
            "TIPO DE MELHORIA": "MELHORIA_SUGERIDA",
            "DESCONEXÕES (GAP)": "GAPS",
            "CAUSA": "CAUSA",
            "MELHORIA/SOLUÇÃO": "SOLUCAO",
            "GANHOS/OBJETIVO": "GANHOS"
}

df_base = df_base.rename(columns=renomear)

# Configuração da API OpenAI
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Conexão com o Weaviate
client_wv8 = weaviate.Client(
    url=os.environ.get("WEAVIATE_URL"),
    auth_client_secret=weaviate.AuthApiKey(
        api_key=os.environ.get("WEAVIATE_API_KEY") 
    ),
    additional_headers={
        "X-OpenAI-Api-Key": openai.api_key,  
    }
)



c:\Users\TiagoVettorazzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\weaviate\__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
C:\Users\TiagoVettorazzi\AppData\Local\Temp\ipykernel_24148\2455868963.py:33: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.

In [22]:
schema = {
    "classes": [
        {
            "class": "DocsOportunidades",
            "description": "Detalhes de melhorias em processos empresariais",
            "properties": [
                {"name": "EMPRESA", "dataType": ["text"], "description": "Nome da empresa"},
                {"name": "SEGMENTO_MERCADO", "dataType": ["text"], "description": "Segmento de mercado"},
                {"name": "PROCESSO", "dataType": ["text"], "description": "Descrição do processo"},
                {"name": "ATIVIDADE_RELACIONADA", "dataType": ["text"], "description": "Atividade relacionada ao processo"},
                {"name": "MELHORIA_SUGERIDA", "dataType": ["text"], "description": "Categoria da melhoria sugerida"},
                {"name": "GAPS", "dataType": ["text"], "description": "Problemas ou lacunas identificadas"},
                {"name": "CAUSA", "dataType": ["text"], "description": "Causa raiz do problema"},
                {"name": "SOLUCAO", "dataType": ["text"], "description": "Proposta de melhoria ou solução"},
                {"name": "GANHOS", "dataType": ["text"], "description": "Ganhos ou objetivos esperados com a solução"}
            ],
            "vectorizer": "text2vec-openai"  # Permite vetorização automática
        }
    ]
}

# Verifica se o schema já existe
if not client_wv8.schema.contains({'classes': [{'class': 'DocsOportunidades'}]}):
    client_wv8.schema.create(schema)



KeyError: 'properties'

In [44]:
lista = [] 
for i in range(len(df_base)):
    lista.append({
                    'EMPRESA': df_base["EMPRESA"][i],
                    'SEGMENTO_MERCADO': df_base["SEGMENTO_MERCADO"][i],
                    'PROCESSO': df_base["PROCESSO"][i], 
                    'ATIVIDADE_RELACIONADA': df_base["ATIVIDADE_RELACIONADA"][i], 
                    'MELHORIA_SUGERIDA': df_base["MELHORIA_SUGERIDA"][i],
                    'GAPS':df_base["GAPS"][i], 
                    'CAUSA' : df_base["CAUSA"][i],
                    'SOLUCAO' : df_base["SOLUCAO"][i],
                    'GANHOS' : df_base["GANHOS"][i]
                 })
    
lista

[{'EMPRESA': 'Marquise',
  'SEGMENTO_MERCADO': 'Ambiental',
  'PROCESSO': 'PROCESSO',
  'ATIVIDADE_RELACIONADA': 'Solicitar tratativa com o fornecedor',
  'MELHORIA_SUGERIDA': 'Características do Produto/Serviço',
  'GAPS': 'As respostas das não conformidades dos fornecedores tem falta de clareza das informações.',
  'CAUSA': 'Fornecedores possuem dificuldade em responder o material enviado.',
  'SOLUCAO': 'Realizar revisão do material de resposta a não conformidades, visando torná-lo mais intuitivo para os fornecedores.',
  'GANHOS': 'Maior qualidade e agilidade na tratativa da não conformidade enviada ao fornecedor.'},
 {'EMPRESA': 'Marquise',
  'SEGMENTO_MERCADO': 'Ambiental',
  'PROCESSO': 'Gerir fornecedores',
  'ATIVIDADE_RELACIONADA': 'Solicitar tratativa com o fornecedor',
  'MELHORIA_SUGERIDA': 'Sistemas / Infraestrutura',
  'GAPS': 'A solicitação de tratativa das não conformidades dos forncedores é realizada via e-mail.',
  'CAUSA': 'Ausência de plataforma oficial para tratat

In [45]:
import requests
import json

try:
    # client.connect()
    client_wv8.batch.configure(batch_size=100)
    with client_wv8.batch as batch:  # Initialize a batch process
        for i, d in enumerate(lista):  # Batch import data
            properties={
            "EMPRESA": d['EMPRESA'],
            "SEGMENTO_MERCADO": d['SEGMENTO_MERCADO'],
            "PROCESSO": d['PROCESSO'],
            "ATIVIDADE_RELACIONADA": d['ATIVIDADE_RELACIONADA'],
            "MELHORIA_SUGERIDA": d['MELHORIA_SUGERIDA'],
            "GAPS": d['GAPS'],
            "CAUSA": d['CAUSA'],
            "SOLUCAO": d['SOLUCAO'],
            "GANHOS": d['GANHOS']
        }
        try:
            batch.add_data_object(
                data_object=properties,class_name="DocsOportunidades"
            )
            print(f"importing question: {i + 1}")
                        
        except Exception as e:
            print(f'Erro ao fazer add_data_object --> {e}')

except Exception as e:
    print(f"ERRO: {e} - INICIO")

# finally:
#     weaviate_client.close()

importing question: 1268


In [9]:
client_wv8.schema.get()

{'classes': [{'class': 'DocsOportunidades',
   'description': 'Detalhes de melhorias em processos empresariais',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'text-embedding-3-small',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'autoTenantActivation': False,
    'autoTenantCreation': False,
    'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'Nome da empresa',
     'indexFilterable': True,
     'indexRangeFilters': False,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'eMPRESA',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'Segmento de mercado',
     'indexFilterable': True,
     'indexRangeFilters': False

In [46]:
import pandas as pd
import openai
import weaviate
import os
import streamlit as st
from dotenv import load_dotenv


df_base = pd.read_excel('Base.xlsx', skiprows = 2, header = 1)
df_base = df_base.iloc[:, :9]
# df_base = df_base.dropna().reset_index(drop=True)
df_base = df_base.fillna("Não informado")


renomear = {"EMPRESA": "EMPRESA",
            "SEGMENTO DE MERCADO": "SEGMENTO_MERCADO",
            "PROCESSO": "PROCESSO",
            "ATIVIDADE RELACIONADA": "ATIVIDADE_RELACIONADA",
            "TIPO DE MELHORIA": "MELHORIA_SUGERIDA",
            "DESCONEXÕES (GAP)": "GAPS",
            "CAUSA": "CAUSA",
            "MELHORIA/SOLUÇÃO": "SOLUCAO",
            "GANHOS/OBJETIVO": "GANHOS"
}

df_base = df_base.rename(columns=renomear)

# Configuração da API OpenAI
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Conexão com o Weaviate
client_wv8 = weaviate.Client(
    url=os.environ.get("WEAVIATE_URL"),
    auth_client_secret=weaviate.AuthApiKey(
        api_key=os.environ.get("WEAVIATE_API_KEY") 
    ),
    additional_headers={
        "X-OpenAI-Api-Key": openai.api_key,  
    }
)


lista = [] 
for i in range(len(df_base)):
    lista.append({
                    'EMPRESA': df_base["EMPRESA"][i],
                    'SEGMENTO_MERCADO': df_base["SEGMENTO_MERCADO"][i],
                    'PROCESSO': df_base["PROCESSO"][i], 
                    'ATIVIDADE_RELACIONADA': df_base["ATIVIDADE_RELACIONADA"][i], 
                    'MELHORIA_SUGERIDA': df_base["MELHORIA_SUGERIDA"][i],
                    'GAPS':df_base["GAPS"][i], 
                    'CAUSA' : df_base["CAUSA"][i],
                    'SOLUCAO' : df_base["SOLUCAO"][i],
                    'GANHOS' : df_base["GANHOS"][i]
                 })

try:
    # client.connect()
    client_wv8.batch.configure(batch_size=100)
    with client_wv8.batch as batch:  # Initialize a batch process
        for i, d in enumerate(lista):  # Batch import data
            properties={
            "EMPRESA": d['EMPRESA'],
            "SEGMENTO_MERCADO": d['SEGMENTO_MERCADO'],
            "PROCESSO": d['PROCESSO'],
            "ATIVIDADE_RELACIONADA": d['ATIVIDADE_RELACIONADA'],
            "MELHORIA_SUGERIDA": d['MELHORIA_SUGERIDA'],
            "GAPS": d['GAPS'],
            "CAUSA": d['CAUSA'],
            "SOLUCAO": d['SOLUCAO'],
            "GANHOS": d['GANHOS']
        }
        try:
            batch.add_data_object(
                data_object=properties,class_name="DocsOportunidades"
            )
            print(f"importing question: {i + 1}")
                        
        except Exception as e:
            print(f'Erro ao fazer add_data_object --> {e}')

except Exception as e:
    print(f"ERRO: {e} - INICIO")




c:\Users\TiagoVettorazzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\weaviate\__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
C:\Users\TiagoVettorazzi\AppData\Local\Temp\ipykernel_24148\3300518586.py:33: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.

In [47]:
import pandas as pd
import openai
import weaviate
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configuração da API OpenAI
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Conexão com o Weaviate
client_wv8 = weaviate.Client(
    url=os.environ.get("WEAVIATE_URL"),
    auth_client_secret=weaviate.AuthApiKey(api_key=os.environ.get("WEAVIATE_API_KEY")),
    additional_headers={
        "X-OpenAI-Api-Key": openai.api_key,
    }
)

# Criar o schema dinamicamente
schema = {
    "classes": [
        {
            "class": "DocsOportunidades",
            "description": "Detalhes de melhorias em processos empresariais",
            "properties": [
                {"name": "EMPRESA", "dataType": ["text"], "description": "Nome da empresa"},
                {"name": "SEGMENTO_MERCADO", "dataType": ["text"], "description": "Segmento de mercado"},
                {"name": "PROCESSO", "dataType": ["text"], "description": "Descrição do processo"},
                {"name": "ATIVIDADE_RELACIONADA", "dataType": ["text"], "description": "Atividade relacionada ao processo"},
                {"name": "MELHORIA_SUGERIDA", "dataType": ["text"], "description": "Categoria da melhoria sugerida"},
                {"name": "GAPS", "dataType": ["text"], "description": "Problemas ou lacunas identificadas"},
                {"name": "CAUSA", "dataType": ["text"], "description": "Causa raiz do problema"},
                {"name": "SOLUCAO", "dataType": ["text"], "description": "Proposta de melhoria ou solução"},
                {"name": "GANHOS", "dataType": ["text"], "description": "Ganhos ou objetivos esperados com a solução"}
            ],
            "vectorizer": "text2vec-openai"
        }
    ]
}

# Verificar se o schema existe, se não, criá-lo
class_name = "DocsOportunidades"
existing_classes = [cls["class"] for cls in client_wv8.schema.get().get("classes", [])]

if class_name not in existing_classes:
    client_wv8.schema.create(schema)
    print(f"Schema '{class_name}' criado com sucesso.")
else:
    print(f"Schema '{class_name}' já existe.")

# Carregar os dados
df_base = pd.read_excel('Base.xlsx', skiprows=2, header=1)
df_base = df_base.iloc[:, :9]
df_base = df_base.fillna("Não informado")

renomear = {
    "EMPRESA": "EMPRESA",
    "SEGMENTO DE MERCADO": "SEGMENTO_MERCADO",
    "PROCESSO": "PROCESSO",
    "ATIVIDADE RELACIONADA": "ATIVIDADE_RELACIONADA",
    "TIPO DE MELHORIA": "MELHORIA_SUGERIDA",
    "DESCONEXÕES (GAP)": "GAPS",
    "CAUSA": "CAUSA",
    "MELHORIA/SOLUÇÃO": "SOLUCAO",
    "GANHOS/OBJETIVO": "GANHOS"
}

df_base = df_base.rename(columns=renomear)

# Preparar os dados para inserção
lista = df_base.to_dict(orient='records')

# Inserir os dados no Weaviate
try:
    client_wv8.batch.configure(batch_size=100)
    with client_wv8.batch as batch:
        for i, d in enumerate(lista):
            properties = {
                "EMPRESA": d['EMPRESA'],
                "SEGMENTO_MERCADO": d['SEGMENTO_MERCADO'],
                "PROCESSO": d['PROCESSO'],
                "ATIVIDADE_RELACIONADA": d['ATIVIDADE_RELACIONADA'],
                "MELHORIA_SUGERIDA": d['MELHORIA_SUGERIDA'],
                "GAPS": d['GAPS'],
                "CAUSA": d['CAUSA'],
                "SOLUCAO": d['SOLUCAO'],
                "GANHOS": d['GANHOS']
            }
            try:
                batch.add_data_object(
                    data_object=properties,
                    class_name="DocsOportunidades"
                )
                print(f"Registro {i + 1} importado com sucesso.")
            except Exception as e:
                print(f"Erro ao adicionar o registro {i + 1}: {e}")
except Exception as e:
    print(f"Erro ao iniciar o batch: {e}")


C:\Users\TiagoVettorazzi\AppData\Local\Temp\ipykernel_24148\3938289980.py:14: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client_wv8 = weaviate.Client(


Schema 'DocsOportunidades' já existe.
Registro 1 importado com sucesso.
Registro 2 importado com sucesso.
Registro 3 importado com sucesso.
Registro 4 importado com sucesso.
Registro 5 importado com sucesso.
Registro 6 importado com sucesso.
Registro 7 importado com sucesso.
Registro 8 importado com sucesso.
Registro 9 importado com sucesso.
Registro 10 importado com sucesso.
Registro 11 importado com sucesso.
Registro 12 importado com sucesso.
Registro 13 importado com sucesso.
Registro 14 importado com sucesso.
Registro 15 importado com sucesso.
Registro 16 importado com sucesso.
Registro 17 importado com sucesso.
Registro 18 importado com sucesso.
Registro 19 importado com sucesso.
Registro 20 importado com sucesso.
Registro 21 importado com sucesso.
Registro 22 importado com sucesso.
Registro 23 importado com sucesso.
Registro 24 importado com sucesso.
Registro 25 importado com sucesso.
Registro 26 importado com sucesso.
Registro 27 importado com sucesso.
Registro 28 importado com 

In [49]:
from weaviate import Client
from weaviate.auth import AuthApiKey

weaviate_url = os.getenv("WEAVIATE_URL")
api_key = os.getenv("WEAVIATE_API_KEY")

auth_config = AuthApiKey(api_key=api_key)
client = Client(url=weaviate_url, auth_client_secret=auth_config)

result = client.data_object.get(class_name='DocsOportunidades', limit=3)
print(result)


C:\Users\TiagoVettorazzi\AppData\Local\Temp\ipykernel_24148\1304121670.py:8: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = Client(url=weaviate_url, auth_client_secret=auth_config)
c:\Users\TiagoVettorazzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\weaviate\warnings.py:186: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated

{'deprecations': [], 'objects': [{'class': 'DocsOportunidades', 'creationTimeUnix': 1736162399051, 'id': '000c1d93-f56b-455c-9335-f7face69b480', 'lastUpdateTimeUnix': 1736162399051, 'properties': {'aTIVIDADE_RELACIONADA': 'Acompanhar Auditoria Interna ou Externa', 'cAUSA': 'Não informado', 'eMPRESA': 'Marista', 'gANHOS': 'Estabelecida uma ferramenta padrão e segura de comunicação com a Auditoria Externa', 'gAPS': 'Não está definida uma ferramenta para realizar comunicação com a Auditoria Externa', 'mELHORIA_SUGERIDA': 'Fluxo de trabalho / Regras de negócio', 'pROCESSO': 'Acompanhar Auditoria Interna ou Externa', 'sEGMENTO_MERCADO': 'Educação', 'sOLUCAO': 'Definir se o PZM será utilizado como ferramenta padrão de comunicação com a auditoria externa'}, 'vectorWeights': None}, {'class': 'DocsOportunidades', 'creationTimeUnix': 1736162420543, 'id': '002a0ef8-e39e-4241-9e11-bdfa3f8e94ff', 'lastUpdateTimeUnix': 1736162420543, 'properties': {'aTIVIDADE_RELACIONADA': 'Solicitar os ajustes e co